# 라이브러리 설치

In [ ]:
!pip install --upgrade pip setuptools wheel

In [ ]:
pip install --no-build-isolation langsmith

In [ ]:
%pip install --user --quiet  google-cloud-aiplatform \
                             google-cloud-storage \
                             google-cloud-pipeline-components \
                             kfp

# 환경 변수 설정

In [ ]:
PROJECT_ID = "ureca-poc-itcen"
LOCATION = "us-central1"
BUCKET_URI = "gs://ureca_test"
PIPELINE_ROOT = f"{BUCKET_URI}/pipeline/exp"
MODEL_URI = f"{PIPELINE_ROOT}/compare-pipeline"
DATASET_ID = "test_data"
TABLE_ID = "amazon_review_v1"


SERVICE_ACCOUNT = "vertex-api@ureca-poc-itcen.iam.gserviceaccount.com"

# 라이브러리 불러오기

In [ ]:
# 일반 라이브러리
import json
import logging
import time
import uuid
from typing import NamedTuple, List

# Vertex AI 라이브러리
from google.cloud import aiplatform as vertex_ai
from google.cloud.aiplatform_v1.types.pipeline_state import PipelineState

# Kubeflow Pipelines(KFP) 관련 라이브러리
from kfp import compiler, dsl, client
from kfp import components
from kfp.dsl import (
    Artifact, Metrics, Dataset, Input, Model, Output, component
)
import kfp.v2.dsl as dsl

# 로깅 설정
logger = logging.getLogger("logger")
logging.basicConfig(level=logging.INFO)

# Vertex AI 초기화
vertex_ai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
# 사전 빌드된 학습용 컨테이너 이미지 URI
TRAIN_IMAGE = vertex_ai.helpers.get_prebuilt_prediction_container_uri(
    framework="sklearn",
    framework_version="1.3",
    accelerator="cpu"
)

# 컴포넌트 파이프라인

In [ ]:
# 데이터 생성 컴포넌트
@component(
    base_image="python:3.10",
    packages_to_install=["pandas", "google-cloud-bigquery", "db-dtypes"]
)

def create_dataset(
    project_id: str,
    dataset_id: str,
    table_id: str,
    in_test_size: float,
    train_data_artifact: Output[Artifact],     # (출력) 학습 데이터
    test_data_artifact: Output[Artifact],      # (출력) 테스트 데이터
    metadata_artifact: Output[Artifact]        # (출력) 사용자/아이템 인덱스 메타데이터
):
    # 라이브러리
    import pandas as pd
    from google.cloud import bigquery
    import os
    import json

    # BigQuery 클라이언트 초기화
    client = bigquery.Client(project=project_id, location="us-central1")

    # 쿼리
    query = f"""
        SELECT customer_id, product_id, star_rating
        FROM `{project_id}.{dataset_id}.{table_id}`
    """

    df = client.query(query).to_dataframe()

    user_ids = {user: i for i, user in enumerate(df['customer_id'].unique())}
    item_ids = {item: i for i, item in enumerate(df['product_id'].unique())}

    df['user_idx'] = df['customer_id'].map(user_ids)
    df['item_idx'] = df['product_id'].map(item_ids)

    train_df = df.sample(frac=1 - in_test_size, random_state=0)
    test_df = df.drop(train_df.index)

    # 학습 데이터 저장
    os.makedirs(train_data_artifact.path, exist_ok=True)
    train_df.to_csv(os.path.join(train_data_artifact.path, "train.csv"), index=False)

    # 테스트 데이터 저장
    os.makedirs(test_data_artifact.path, exist_ok=True)
    test_df.to_csv(os.path.join(test_data_artifact.path, "test.csv"), index=False)

    # 메타데이터 저장
    os.makedirs(metadata_artifact.path, exist_ok=True)
    with open(os.path.join(metadata_artifact.path, "ids.json"), 'w') as f:
        json.dump({
            "user_ids": user_ids,
            "item_ids": item_ids
        }, f)

# 컴포넌트 YAML 파일로 컴파일
compiler.Compiler().compile(create_dataset, "create_dataset.yaml")


In [ ]:
# 모델 학습 컴포넌트
@component(
    base_image="python:3.10",
    packages_to_install=["numpy", "pandas", "joblib"]
)

def train_model(
    train_data: Input[Artifact],           # (입력) 학습 데이터셋
    metadata: Input[Artifact],             # (입력) 메타데이터
    n_factors: int,
    learning_rate: float,
    reg: float,
    num_epochs: int,
    model_artifact: Output[Model]          # (출력) 모델
):
    # 라이브러리
    import numpy as np
    import pandas as pd
    import os
    import json
    import joblib
    from collections import defaultdict

    # 데이터 불러오기
    df = pd.read_csv(os.path.join(train_data.path, "train.csv"))
    with open(os.path.join(metadata.path, "ids.json")) as f:
        ids = json.load(f)

    num_users = len(ids['user_ids'])
    num_items = len(ids['item_ids'])

    P = np.random.normal(0, 0.1, (num_users, n_factors))
    Q = np.random.normal(0, 0.1, (num_items, n_factors))

    # 학습
    for epoch in range(num_epochs):
        for _, row in df.iterrows():
            u, i, r = row['user_idx'], row['item_idx'], row['star_rating']
            pred = np.dot(P[u], Q[i].T)
            err = r - pred

            # 파라미터 업데이트
            P[u] += learning_rate * (err * Q[i] - reg * P[u])
            Q[i] += learning_rate * (err * P[u] - reg * Q[i])

    # 학습된 행렬과 메타데이터 저장
    os.makedirs(model_artifact.path, exist_ok=True)
    np.save(os.path.join(model_artifact.path, "P.npy"), P)
    np.save(os.path.join(model_artifact.path, "Q.npy"), Q)
    joblib.dump(ids, os.path.join(model_artifact.path, "metadata.joblib"))

# 컴포넌트 YAML 파일로 컴파일
compiler.Compiler().compile(train_model, "train_model.yaml")


In [ ]:
# 모델 평가 컴포넌트
@component(
    base_image="python:3.10",
    packages_to_install=["numpy", "pandas", "joblib", "scikit-learn"]
)

def evaluate_model(
    test_data: Input[Artifact],             # (입력) 테스트 데이터셋
    model: Input[Model],                    # (입력) 학습된 모델
    evaluate_artifact: Output[Artifact],    # (출력) 평가 결과
    metrics: Output[Metrics]
):
    # 라이브러리
    import numpy as np
    import pandas as pd
    import joblib
    import os
    import json
    from sklearn.metrics import mean_squared_error

    # 테스트 데이터 불러오기
    df = pd.read_csv(os.path.join(test_data.path, "test.csv"))

    # 학습된 행렬 불러오기
    P = np.load(os.path.join(model.path, "P.npy"))
    Q = np.load(os.path.join(model.path, "Q.npy"))
    ids = joblib.load(os.path.join(model.path, "metadata.joblib"))

    # 예측
    predictions = [np.dot(P[row['user_idx']], Q[row['item_idx']]) for _, row in df.iterrows()]

    # 평가지표
    rmse = np.sqrt(mean_squared_error(df['star_rating'], predictions))

    # 평가지표 저장
    metrics.log_metric("RMSE", rmse)

    # 평가 결과 JSON으로 저장
    with open(os.path.join(evaluate_artifact.path, "metrics.json"), 'w') as f:
        json.dump({"RMSE": rmse}, f, indent=4)

# 컴포넌트를 YAML로 컴파일
compiler.Compiler().compile(evaluate_model, "evaluate_model.yaml")


In [ ]:
# 분리된 컴포넌트 YAML 파일 로드
create_dataset_comp = components.load_component_from_file("create_dataset.yaml")
train_model_comp = components.load_component_from_file("train_model.yaml")
evaluate_model_comp = components.load_component_from_file("evaluate_model.yaml")

# 파이프라인 정의
@dsl.pipeline(
    name="svd_separated_pipeline",
    description="SVD 기반 추천 모델을 학습하고 평가하는 파이프라인"
)

def pipeline(
    project_id: str,
    dataset_id: str,
    table_id: str,
    model_uri: str,
    n_factors: int,
    learning_rate: float,
    reg: float,
    num_epochs: int,
    in_test_size: float
):

    # 데이터 생성 및 전처리 컴포넌트 실행
    dataset_task = create_dataset_comp(
        project_id=project_id,
        dataset_id=dataset_id,
        table_id=table_id,
        in_test_size=in_test_size
    )

    # 모델 학습 컴포넌트 실행
    train_task = train_model_comp(
        train_data=dataset_task.outputs["train_data_artifact"],
        metadata=dataset_task.outputs["metadata_artifact"],
        n_factors=n_factors,
        learning_rate=learning_rate,
        reg=reg,
        num_epochs=num_epochs
    )

    # 모델 평가 컴포넌트 실행
    evaluate_model_comp(
        test_data=dataset_task.outputs["test_data_artifact"],
        model=train_task.outputs["model_artifact"]
    )

# 파이프라인 컴파일
compiler.Compiler().compile(pipeline_func=pipeline, package_path="svd_pipeline.json")

In [ ]:
# 실험 파라미터 리스트 설정
runs = [
    {"n_factors": 10, "learning_rate": 0.01, "reg": 0.02, "num_epochs": 50, "in_test_size": 0.2},
    {"n_factors": 50, "learning_rate": 0.01, "reg": 0.1, "num_epochs": 50, "in_test_size": 0.2},
    {"n_factors": 100, "learning_rate": 0.01, "reg": 0.01, "num_epochs": 50, "in_test_size": 0.2},
]

In [ ]:
# 실험 설정
EXPERIMENT_NAME = "svd-comparing-exp"

# 파라미터에 대해 파이프라인 반복 실행
for i, run in enumerate(runs):

    job = vertex_ai.PipelineJob(
        display_name=f"{EXPERIMENT_NAME}-pipeline-run-{i}",
        template_path="svd_pipeline.json",
        pipeline_root=PIPELINE_ROOT,
        location="us-central1",

        parameter_values={
            "project_id": PROJECT_ID,
            "dataset_id": DATASET_ID,
            "table_id": TABLE_ID,
            "model_uri": MODEL_URI,
            **run,
        },
    )

    job.submit(experiment=EXPERIMENT_NAME)
